In [36]:
import pandas as pd
import numpy as np
import re

In [37]:
defending_df = pd.read_csv('uncleaned_data_csv/defending.csv', header=None)
passing_df = pd.read_csv('uncleaned_data_csv/passing.csv', header=None)
seasons_stats_df = pd.read_csv('uncleaned_data_csv/seasons_stats.csv', header=None)
standard_df = pd.read_csv('uncleaned_data_csv/standard.csv', header=None)
shooting_df = pd.read_csv('uncleaned_data_csv/shooting.csv', header=None)
seasons_wages_df = pd.read_csv('uncleaned_data_csv/seasons_wages.csv')

## Cleaning defending_df

In [38]:
defending_df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NaN,NaN,NaN,NaN,NaN,NaN,Rk,Player,Nation,Position,...,Tackles (Def 3rd),Tackles (Mid 3rd),Tackles (Att 3rd),Dribblers Tackled,Dribbles Challenged,% of Dribblers Tackled,Challenges Lost,Blocks,Shots Blocked,Season
1,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,26,1990,24.3,47,...,18,24,5,19,47,94,64,2,Matches,2017-2018
2,2,Rolando Aarons,eng ENG,"MF,FW",Newcastle Utd,eng Premier League,21,1995,1.5,4,...,2,3,0,3,1,5,0,0,Matches,2017-2018
3,3,Rolando Aarons,eng ENG,"MF,FW",Hellas Verona,it Serie A,21,1995,5.7,13,...,6,3,1,2,2,15,0,0,Matches,2017-2018
4,4,Ignazio Abate,it ITA,DF,Milan,it Serie A,30,1986,11.7,20,...,5,22,2,20,8,28,29,0,Matches,2017-2018


## Understanding the values in each column

In [39]:
for col in defending_df.columns:
    print(f"Column: {col}")
    print(defending_df[col].unique()[:10])  # Print first 10 unique values
    print("\n")

Column: 0
[nan '1' '2' '3' '4' '5' '6' '7' '8' '9']


Column: 1
[nan 'Patrick van Aanholt' 'Rolando Aarons' 'Ignazio Abate'
 'Aymen Abdennour' 'Aly Abeid' 'Mehdi Abeid' 'David Abraham'
 'Tammy Abraham' 'Amir Abrashi']


Column: 2
[nan 'nl NED' 'eng ENG' 'it ITA' 'tn TUN' 'mr MTN' 'dz ALG' 'ar ARG'
 'al ALB' 'gh GHA']


Column: 3
[nan 'DF' 'MF,FW' 'MF' 'FW' 'GK' 'DF,FW' 'Pos' 'DF,MF' 'FW,MF']


Column: 4
[nan 'Crystal Palace' 'Newcastle Utd' 'Hellas Verona' 'Milan' 'Marseille'
 'Levante' 'Dijon' 'Eint Frankfurt' 'Swansea City']


Column: 5
[nan 'eng Premier League' 'it Serie A' 'fr Ligue 1' 'es La Liga'
 'de Bundesliga' 'Comp']


Column: 6
['Rk' '26' '21' '30' '27' '19' '24' '31' '29' '25']


Column: 7
['Player' '1990' '1995' '1986' '1989' '1997' '1992' '1988' '1985' '1987']


Column: 8
['Nation' '24.3' '1.5' '5.7' '11.7' '5.5' '0.9' '13.1' '25.6' '19.2']


Column: 9
['Position' '47' '4' '13' '20' '7' '2' '23' '40' '10']


Column: 10
['Squad' '32' '4' '8' '17' '3' '1' '15' '6' '5']


Co

## Fixing column names in the table

In [40]:
def clean_header_rows_modular(df, header_identifier="Rk"):
    """
    Cleans a DataFrame by:
    1. Removing the first row (incorrect header).
    2. Finding the first row where the first column matches header_identifier (e.g., 'Rk') and using it as the header.
    3. Removing all rows where the first column matches header_identifier (removes repeated headers).
    4. Resets the index and returns the cleaned DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        header_identifier (str): The value in the first column that identifies the header row.

    Returns:
        pd.DataFrame: Cleaned DataFrame with proper headers and no repeated header rows.
    """
    # 1. Remove the first row (incorrect header)
    df = df.drop(index=0).reset_index(drop=True)

    # 2. Find the first row where the first column matches header_identifier
    header_row_idx = df[df.iloc[:, 0] == header_identifier].index
    if len(header_row_idx) == 0:
        raise ValueError(f"No header row found with first column == '{header_identifier}'")
    header_row_idx = header_row_idx[0]

    # 3. Set the header
    new_header = df.iloc[header_row_idx]
    df = df[(df.iloc[:, 0] != header_identifier)].reset_index(drop=True)
    df.columns = new_header

    # 4. Remove the header row from the data (if still present)
    df = df[df.iloc[:, 0] != header_identifier].reset_index(drop=True)

    # 5. Check: Ensure no header rows remain
    assert (df.iloc[:, 0] != header_identifier).all(), "Header rows still present after cleaning!"

    return df

In [41]:
# # View unique entries in the 'Nation' column
# print("Unique values in the Nation column:")
# print(defending_df_cleaned['Nation'].unique())

# # View unique entries in the 'Comp' column
# print("\nUnique values in the Comp column:")
# print(defending_df_cleaned['Comp'].unique())

## Version that is modular and can be used for any df. Column names must be supplied

In [42]:
def rename_duplicate_columns_prime(df, col_name, new_names=None):
    """
    Renames duplicate columns to ensure uniqueness.
    By default, appends _1, _2, etc. to duplicates, or uses new_names if provided.

    Parameters:
        df (pd.DataFrame): The DataFrame with potential duplicate columns.
        col_name (str): The column name you expect to be duplicated.
        new_names (list, optional): List of new names for each duplicate (must match number of duplicates).

    Returns:
        pd.DataFrame: Updated DataFrame with unique column names.
    """
    columns = list(df.columns)
    indices = [i for i, col in enumerate(columns) if col == col_name]
    if len(indices) > 1:
        if new_names and len(new_names) == len(indices):
            for idx, new_col in zip(indices, new_names):
                df.columns.values[idx] = new_col
        else:
            # Keep the first as is, rename the rest with suffixes
            for i, idx in enumerate(indices[1:], 1):
                df.columns.values[idx] = f"{col_name}_{i}"
    return df

In [43]:
# defending_df_cleaned = clean_header_rows_modular(defending_df)
# defending_df_cleaned.head()

In [44]:
# for col in defending_df_cleaned.columns:
#     print(f"Column: {col}")
#     print(defending_df_cleaned[col].unique()[:10])  # Print first 10 unique values
#     print("\n")

In [45]:
def keep_after_first_capital(s):
    """
    Extracts the substring starting from the first uppercase letter in a string.
    If the input is not a string, it returns an empty string.
    """
    if not isinstance(s, str):  # Check if the input is not a string
        return ''
    
    # Regular expression to match from the first uppercase letter to the end
    match = re.search(r'[A-Z].*', s)
    if match:
        return match.group(0)
    return ''

In [46]:
# # Apply the function to 'Nation' and 'Comp' columns
# defending_df_cleaned['Nation'] = defending_df_cleaned['Nation'].apply(keep_after_first_capital)
# defending_df_cleaned['Comp'] = defending_df_cleaned['Comp'].apply(keep_after_first_capital)

# # Display the cleaned columns
# print(defending_df_cleaned[['Nation', 'Comp']].head(15))

In [47]:
def extract_clean_names_generic(df, columns_to_clean):
    """
    Cleans specified columns in a DataFrame by extracting the substring
    starting from the first uppercase letter.

    Parameters:
        df (pd.DataFrame): The DataFrame to clean.
        columns_to_clean (list): List of column names to clean (e.g., ['Squad', 'Comp', 'Nation']).

    Returns:
        pd.DataFrame: Updated DataFrame with cleaned columns.
    """
    def keep_after_first_capital(s):
        if not isinstance(s, str):
            return ''
        match = re.search(r'[A-Z].*', s)
        return match.group(0) if match else ''

    for col in columns_to_clean:
        if col in df.columns:
            df[col] = df[col].apply(keep_after_first_capital)
    return df

# Usage examples:
# For most tables: extract_clean_names_generic(df, ['Squad', 'Comp'])
# For tables with Nation: extract_clean_names_generic(df, ['Squad', 'Comp', 'Nation'])

In [48]:
def convert_all_numeric_except(df, exclude_cols):
    """
    Converts all columns in the DataFrame to float except those in exclude_cols.
    'Born' is converted to Int64 if present.
    """
    for col in df.columns:
        if col == 'Born':
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        elif col not in exclude_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)
    return df

In [49]:
def normalize_metrics_prime(df, per90_columns):
    """
    Normalizes specified columns per 90 minutes.

    Parameters:
        df (pd.DataFrame): The DataFrame containing stats.
        per90_columns (dict): Keys are columns to normalize, values are new column names.

    Returns:
        pd.DataFrame: Updated DataFrame with normalized metrics.
    """
    for col, new_col in per90_columns.items():
        if col in df.columns:
            df[new_col] = (df[col] / df['90s']).round(2)
    return df

In [50]:
def process_data_prime(
    file_path,
    header_identifier="Rk",
    columns_to_clean=None,
    duplicate_columns_info=None,
    per90_columns=None,
    min_90s=0,
    exclude_cols=None
):
    """
    Master function to process and clean the dataset.

    Parameters:
        file_path (str): Path to the CSV file.
        header_identifier (str): Value in the first column that identifies the header row (default 'Rk').
        columns_to_clean (list): Columns to clean with extract_clean_names_generic (e.g., ['Squad', 'Comp', 'Nation']).
        duplicate_columns_info (dict, optional): 
            Keys are column names you expect to be duplicated.
            Values are lists of new names for those columns (or None to use default suffixes).
        per90_columns (dict, optional): Columns to normalize per 90 minutes.
        min_90s (int, optional): Minimum 90s threshold for filtering.

    Returns:
        pd.DataFrame: Fully cleaned and processed DataFrame.
    """
    # 1. Read and clean header rows
    df = pd.read_csv(file_path, header=None)
    df = clean_header_rows_modular(df, header_identifier)

    # 2. Rename duplicate columns if needed
    if duplicate_columns_info:
        for col_name, new_names in duplicate_columns_info.items():
            df = rename_duplicate_columns_prime(df, col_name, new_names)

    # 3. Clean text columns (Squad, Comp, Nation, etc.)
    if columns_to_clean:
        df = extract_clean_names_generic(df, columns_to_clean)
    else:
        raise ValueError("columns_to_clean must be provided (e.g., ['Squad', 'Comp'])")
    
    #3.5. Adding season column if not present
    df.columns = list(df.columns[:-1]) + ["Season"]
    
    # 3.6. Drop 'Matches' column if present
    if "Matches" in df.columns:
        df = df.drop(columns=["Matches"])

    # 4. Convert types: all numeric except Born (Int64)
    df = convert_all_numeric_except(df, exclude_cols)

    # 5. Filter by min_90s if column exists
    if '90s' in df.columns:
        df = df[df['90s'] >= min_90s]
    else:
        raise KeyError("'90s' column not found in DataFrame after cleaning.")

    # 6. Normalize metrics if requested
    if per90_columns:
        df = normalize_metrics_prime(df, per90_columns)

    # 7. Final error checks
    if not df.columns.is_unique:
        raise ValueError("DataFrame columns are not unique after processing.")
    if 'Season' not in df.columns:
        raise ValueError("'Season' column missing after processing.")

    return df

# Example usage for passing:
# passing_df_cleaned = process_data_prime(
#     "uncleaned_data_csv/passing.csv",
#     header_identifier="Rk",
#     columns_to_clean=['Squad', 'Comp', 'Nation'],
#     duplicate_columns_info=duplicate_columns_passing,
#     per90_columns=per90_cols_passing,
#     min_90s=5
# )

In [51]:
defending_exclude = [
    'Player', 'Nation', 'Position', 'Pos', 'Squad', 'Comp',
    'Matches', 'Season'
]

duplicate_columns_def = {"Tkl": ["Tkl", "DribTkl"]}

per90_cols_defending = {
    "Int": "Interceptions/90",
    "Tkl": "Tackles/90",
}

In [52]:
defending_df_cleaned = process_data_prime(
    "uncleaned_data_csv/defending.csv",
    header_identifier="Rk",
    columns_to_clean=['Squad', 'Comp', 'Nation'],
    duplicate_columns_info=duplicate_columns_def,
    exclude_cols = defending_exclude,
    per90_columns=per90_cols_defending,
    min_90s=5
)

# Impute Tkl% NaNs with 0 (defending-specific fix)
if 'Tkl%' in defending_df_cleaned.columns:
    defending_df_cleaned['Tkl%'] = defending_df_cleaned['Tkl%'].fillna(0)

defending_df_cleaned.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tkl,...,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Season,Interceptions/90,Tackles/90
0,1.0,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,26.0,1990,24.3,47.0,...,24.0,5.0,19.0,47.0,94.0,64.0,2.0,2017-2018,1.93,1.93
2,3.0,Rolando Aarons,ENG,"MF,FW",Hellas Verona,Serie A,21.0,1995,5.7,13.0,...,3.0,1.0,2.0,2.0,15.0,0.0,0.0,2017-2018,0.35,2.28
3,4.0,Ignazio Abate,ITA,DF,Milan,Serie A,30.0,1986,11.7,20.0,...,22.0,2.0,20.0,8.0,28.0,29.0,0.0,2017-2018,0.68,1.71
4,5.0,Aymen Abdennour,TUN,DF,Marseille,Ligue 1,27.0,1989,5.5,7.0,...,5.0,5.0,0.0,4.0,11.0,20.0,0.0,2017-2018,0.73,1.27
6,7.0,Mehdi Abeid,ALG,MF,Dijon,Ligue 1,24.0,1992,13.1,23.0,...,11.0,6.0,5.0,18.0,41.0,25.0,2.0,2017-2018,1.37,1.76


In [53]:
# Fill missing data for Marco Pellegrino in defending_df_cleaned
mask = (defending_df_cleaned['Player'] == 'Marco Pellegrino') & (defending_df_cleaned['Squad'] == 'Salernitana')
defending_df_cleaned.loc[mask, 'Born'] = 2002
defending_df_cleaned.loc[mask, 'Nation'] = 'ARG'

# Optionally, fill Age if you want (e.g., 2024 - 2002 = 22)
defending_df_cleaned.loc[mask, 'Age'] = 22

## Cleaning passing_df

In [54]:
passing_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rk,...,Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Assists,Season
1,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,26,1990,24.3,884,...,2.1,1.8,-1.1,18,63,28,6,92,Matches,2017-2018
2,2,Rolando Aarons,eng ENG,"MF,FW",Newcastle Utd,eng Premier League,21,1995,1.5,29,...,0.0,0.0,0.0,0,2,1,1,3,Matches,2017-2018
3,3,Rolando Aarons,eng ENG,"MF,FW",Hellas Verona,it Serie A,21,1995,5.7,87,...,0.2,0.1,-0.2,3,8,7,1,17,Matches,2017-2018
4,4,Ignazio Abate,it ITA,DF,Milan,it Serie A,30,1986,11.7,625,...,0.5,0.8,-0.5,10,55,20,7,81,Matches,2017-2018


In [55]:
passing_exclude = [
    'Player', 'Nation', 'Position', 'Pos', 'Squad', 'Competition', 'Comp',
    'Matches', 'Season'
]

duplicate_columns_passing = {
    "Cmp":   ["Cmp_Total", "Cmp_Short", "Cmp_Medium", "Cmp_Long"],
    "Att":   ["Att_Total", "Att_Short", "Att_Medium", "Att_Long"],
    "Cmp%":  ["Cmp%_Total", "Cmp%_Short", "Cmp%_Medium", "Cmp%_Long"]
}

per90_cols_passing = {
    "Cmp_Total": "Cmp_Total/90",
    "Cmp_Short": "Cmp_Short/90",
    "Cmp_Medium": "Cmp_Medium/90",
    "Cmp_Long": "Cmp_Long/90",
    "Att_Total": "Att_Total/90",
    "Att_Short": "Att_Short/90",
    "Att_Medium": "Att_Medium/90",
    "Att_Long": "Att_Long/90"
}

In [56]:
# Example usage for passing:
passing_df_cleaned = process_data_prime(
    "uncleaned_data_csv/passing.csv",
    header_identifier="Rk",
    columns_to_clean=['Squad', 'Comp', 'Nation'],
    duplicate_columns_info=duplicate_columns_passing,
    exclude_cols=passing_exclude,
    per90_columns=per90_cols_passing,
    min_90s=5
)
passing_df_cleaned.rename(columns={"1/3": "PassesInFinal3rd"}, inplace=True)
passing_df_cleaned.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Cmp_Total,...,PrgP,Season,Cmp_Total/90,Cmp_Short/90,Cmp_Medium/90,Cmp_Long/90,Att_Total/90,Att_Short/90,Att_Medium/90,Att_Long/90
0,1.0,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,26.0,1990,24.3,884.0,...,92.0,2017-2018,36.38,19.71,12.80,3.05,48.40,21.89,17.74,6.26
2,3.0,Rolando Aarons,ENG,"MF,FW",Hellas Verona,Serie A,21.0,1995,5.7,87.0,...,17.0,2017-2018,15.26,8.77,4.39,0.70,21.05,10.53,6.14,1.40
3,4.0,Ignazio Abate,ITA,DF,Milan,Serie A,30.0,1986,11.7,625.0,...,81.0,2017-2018,53.42,23.33,24.53,4.96,66.32,25.13,28.46,10.26
4,5.0,Aymen Abdennour,TUN,DF,Marseille,Ligue 1,27.0,1989,5.5,310.0,...,12.0,2017-2018,56.36,22.91,26.91,4.36,60.55,23.27,28.18,6.18
6,7.0,Mehdi Abeid,ALG,MF,Dijon,Ligue 1,24.0,1992,13.1,513.0,...,49.0,2017-2018,39.16,17.63,16.72,3.36,46.18,20.15,18.70,4.73


In [57]:
passing_df_cleaned.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'Cmp_Total', 'Att_Total', 'Cmp%_Total', 'TotDist', 'PrgDist',
       'Cmp_Short', 'Att_Short', 'Cmp%_Short', 'Cmp_Medium', 'Att_Medium',
       'Cmp%_Medium', 'Cmp_Long', 'Att_Long', 'Cmp%_Long', 'Ast', 'xAG', 'xA',
       'A-xAG', 'KP', 'PassesInFinal3rd', 'PPA', 'CrsPA', 'PrgP', 'Season',
       'Cmp_Total/90', 'Cmp_Short/90', 'Cmp_Medium/90', 'Cmp_Long/90',
       'Att_Total/90', 'Att_Short/90', 'Att_Medium/90', 'Att_Long/90'],
      dtype='object')

In [58]:
# Fill missing data for Marco Pellegrino
mask = (passing_df_cleaned['Player'] == 'Marco Pellegrino') & (passing_df_cleaned['Squad'] == 'Salernitana')
passing_df_cleaned.loc[mask, 'Born'] = 2002
passing_df_cleaned.loc[mask, 'Nation'] = 'ARG'

# Optionally, fill Age if you want (e.g., 2024 - 2002 = 22)
passing_df_cleaned.loc[mask, 'Age'] = 22

In [59]:
shooting_exclude = [
    'Player', 'Nation', 'Position', 'Pos', 'Squad', 'Competition', 'Comp',
    'Matches', 'Season'
]


per90_cols_shooting = {
    "Goals": "Goals_per90",
    "Shots Total": "Shots_Total_per90",
    "Shots on Target": "Shots_on_Target_per90",
    "xG: Expected Goals": "xG_per90",
    "npxG: Non-Penalty xG": "npxG_per90"
}

In [60]:
shooting_df_cleaned = process_data_prime(
    "uncleaned_data_csv/shooting.csv",
    header_identifier="Rk",
    columns_to_clean=['Squad', 'Comp', 'Nation'],
    duplicate_columns_info=None,
    exclude_cols = shooting_exclude,
    per90_columns=per90_cols_shooting,
    min_90s=5
)

shooting_df_cleaned.head()


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Season
0,1.0,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,26.0,1990,24.3,5.0,...,23.4,4.0,0.0,0.0,3.1,3.1,0.09,1.9,1.9,2017-2018
2,3.0,Rolando Aarons,ENG,"MF,FW",Hellas Verona,Serie A,21.0,1995,5.7,0.0,...,17.4,0.0,0.0,0.0,0.2,0.2,0.05,-0.2,-0.2,2017-2018
3,4.0,Ignazio Abate,ITA,DF,Milan,Serie A,30.0,1986,11.7,1.0,...,17.1,0.0,0.0,0.0,0.2,0.2,0.04,0.8,0.8,2017-2018
4,5.0,Aymen Abdennour,TUN,DF,Marseille,Ligue 1,27.0,1989,5.5,0.0,...,11.5,0.0,0.0,0.0,0.1,0.1,0.06,-0.1,-0.1,2017-2018
6,7.0,Mehdi Abeid,ALG,MF,Dijon,Ligue 1,24.0,1992,13.1,0.0,...,25.2,2.0,0.0,0.0,1.0,1.0,0.05,-1.0,-1.0,2017-2018


## Cleaning seasons_df

In [61]:
seasons_stats_df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,NaN,NaN,NaN,NaN,NaN,NaN,Rk,Squad,Competition,# of Players,...,npxG: Non-Penalty xG,xAG: Exp. Assisted Goals,npxG + xAG,Progressive Carries,Progressive Passes,Goals/90,Assists/90,Goals + Assists/90,Non-Penalty Goals/90,Season
1,1.0,Alavés,es La Liga,30.0,25.5,40.3,38,418,"3,420",38.0,...,0.82,1.87,1.03,1.84,1.01,0.73,1.74,0.95,1.68,2017-2018
2,2.0,Amiens,fr Ligue 1,30.0,27.5,43.3,38,418,"3,420",38.0,...,0.63,1.58,0.87,1.50,0.86,0.56,1.42,0.78,1.35,2017-2018
3,3.0,Angers,fr Ligue 1,27.0,27.1,45.1,38,418,"3,420",38.0,...,0.79,1.87,0.97,1.76,1.25,0.93,2.18,1.17,2.10,2017-2018
4,4.0,Arsenal,eng Premier League,30.0,26.8,61.4,38,418,"3,420",38.0,...,1.61,3.53,1.82,3.42,1.80,1.40,3.20,1.69,3.10,2017-2018


In [62]:
def process_seasons_modular(
    df,
    headers,
    columns_to_clean=None,
    exclude_numeric=None,
    columns_to_drop=None,
    min_90s=0,
    ninety_col='90s'
):
    """
    Modular cleaning for FBref tables with custom headers and options.
    """
    # 1. Drop first row and assign headers
    df = df.drop(0).reset_index(drop=True)
    df.columns = headers

    # 2. Rename Competition to Comp if present
    if 'Competition' in df.columns:
        df = df.rename(columns={'Competition': 'Comp'})

    # 3. Drop columns if needed
    if columns_to_drop:
        df = df.drop(columns=columns_to_drop)

    # 4. Clean text columns
    if columns_to_clean:
        df = extract_clean_names_generic(df, columns_to_clean)

    # 5. Clean 'Min' column: remove commas before numeric conversion
    if 'Min' in df.columns:
        df['Min'] = (
            df['Min']
            .astype(str)
            .str.replace(',', '', regex=False)
            .replace('', '0')
            .astype(float)
        )

    # 6. Convert all columns to numeric except exclude list
    if exclude_numeric is not None:
        df = convert_all_numeric_except(df, exclude_numeric)

    # 7. Filter by min_90s if column exists
    if ninety_col in df.columns:
        df = df[df[ninety_col] >= min_90s]
        
    # Impute missing values in 'CrdR' (Red Cards) with 0
    if 'CrdR' in df.columns:
        df['CrdR'] = df['CrdR'].fillna(0)

    return df

In [63]:
seasons_headers = [
    "Rk", "Squad", "Comp", "# Pl", "Age", "Poss", "MP", "Starts", "Min", "90s",
    "Gls", "Ast", "G+A", "G-PK", "PK", "PKatt", "CrdY", "CrdR",
    "xG", "npxG", "xAG", "npxG+xAG",
    "PrgC", "PrgP",
    "Gls_per90", "Ast_per90", "G+A_per90", "G-PK_per90", "G+A-PK_per90",
    "xG_per90", "xAG_per90", "xG+xAG_per90", "npxG_per90", "npxG+xAG_per90",
    "Season"
]

seasons_exclude = ['Rk', 'Squad', 'Comp', 'Season', 'Min']

In [64]:
seasons_stats_cleaned = process_seasons_modular(
    seasons_stats_df,
    seasons_headers,
    columns_to_clean=['Squad', 'Comp'],
    exclude_numeric=seasons_exclude,
    columns_to_drop=None,
    min_90s=0,
    ninety_col=''
)

seasons_stats_cleaned.head()

,Rk,Squad,Comp,# Pl,Age,Poss,MP,Starts,Min,90s,...,Ast_per90,G+A_per90,G-PK_per90,G+A-PK_per90,xG_per90,xAG_per90,xG+xAG_per90,npxG_per90,npxG+xAG_per90,Season
0,1.0,Alavés,La Liga,30.0,25.5,40.3,38.0,418.0,3420.0,38.0,...,0.82,1.87,1.03,1.84,1.01,0.73,1.74,0.95,1.68,2017-2018
1,2.0,Amiens,Ligue 1,30.0,27.5,43.3,38.0,418.0,3420.0,38.0,...,0.63,1.58,0.87,1.50,0.86,0.56,1.42,0.78,1.35,2017-2018
2,3.0,Angers,Ligue 1,27.0,27.1,45.1,38.0,418.0,3420.0,38.0,...,0.79,1.87,0.97,1.76,1.25,0.93,2.18,1.17,2.10,2017-2018
3,4.0,Arsenal,Premier League,30.0,26.8,61.4,38.0,418.0,3420.0,38.0,...,1.61,3.53,1.82,3.42,1.80,1.40,3.20,1.69,3.10,2017-2018
4,5.0,Atalanta,Serie A,25.0,25.7,55.4,38.0,418.0,3420.0,38.0,...,1.00,2.50,1.37,2.37,1.69,1.18,2.86,1.50,2.68,2017-2018


## Cleaning standard_df

In [65]:
standard_df.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rk,Player,Nation,...,xAG: Exp. Assisted Goals,npxG + xAG,Progressive Carries,Progressive Passes,Progressive Passes Rec,Goals/90,Assists/90,Goals + Assists/90,Non-Penalty Goals/90,Season
1,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,26,1990,28,25,...,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,Matches,2017-2018
2,2,Rolando Aarons,eng ENG,"MF,FW",Newcastle Utd,eng Premier League,21,1995,4,1,...,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,Matches,2017-2018
3,3,Rolando Aarons,eng ENG,"MF,FW",Hellas Verona,it Serie A,21,1995,11,6,...,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06,Matches,2017-2018
4,4,Ignazio Abate,it ITA,DF,Milan,it Serie A,30,1986,17,11,...,0.09,0.09,0.09,0.01,0.04,0.06,0.01,0.06,Matches,2017-2018


In [66]:
def remove_internal_header_rows(df, header_col="Rk"):
    """
    Removes any rows where the first column equals the header_col value (e.g., 'Rk'),
    except for the actual header row.
    """
    # Assumes the header has already been set correctly
    return df[df[header_col] != header_col].reset_index(drop=True)

In [67]:
def clean_and_assign_headers(
    df,
    headers,
    columns_to_clean=None,
    drop_columns=None,
    exclude_numeric=None
):
    """
    Cleans a DataFrame by:
    1. Dropping the first row (scraped header).
    2. Assigning custom headers.
    3. Removing internal header rows (rows where 'Rk' == 'Rk').
    4. Dropping extra columns if present.
    5. Cleaning specified text columns.
    6. Optionally dropping columns.
    7. Optionally converting numeric columns.
    """
    # 1. Drop the first row (scraped header)
    df = df.drop(0).reset_index(drop=True)

    # 2. Drop extra columns if present
    if df.shape[1] > len(headers):
        df = df.iloc[:, :len(headers)]

    # 3. Assign custom headers
    df.columns = headers

    # 4. Remove internal header rows (rows where 'Rk' == 'Rk')
    df = remove_internal_header_rows(df, header_col="Rk")

    # 5. Clean specified text columns
    if columns_to_clean:
        df = extract_clean_names_generic(df, columns_to_clean)

    # 6. Drop columns if needed
    if drop_columns:
        df = df.drop(columns=[col for col in drop_columns if col in df.columns])
        
    # 6.5. Clean 'Min' column: remove commas before numeric conversion
    if 'Min' in df.columns:
        df['Min'] = (
            df['Min']
            .astype(str)
            .str.replace(',', '', regex=False)
            .replace('', '0')
            .astype(float)
        )

    # 7. Convert all columns to numeric except those in exclude_numeric
    if exclude_numeric is not None:
        df = convert_all_numeric_except(df, exclude_numeric)
        
    # 8. Impute NaNs with 0 for all columns except 'Born' and 'Age'
    cols_to_impute = [col for col in df.columns if col not in ['Born', 'Age']]
    df[cols_to_impute] = df[cols_to_impute].fillna(0)

    return df

In [68]:
standard_headers = [
    "Rk", "Player", "Nation", "Pos", "Squad", "Comp", "Age", "Born", "MP",
    "Starts", "Min", "90s", "Gls", "Ast", "G+A", "G-PK", "PK", "PKatt",
    "CrdY", "CrdR", "xG", "npxG", "xAG", "npxG+xAG", "PrgC", "PrgP", "PrgR",
    "xG_per90", "xAG_per90", "xG+xAG_per90", "npxG_per90", "npxG+xAG_per90",
    "Gls_per90", "Ast_per90", "G+A_per90", "G-PK_per90", "G+A-PK_per90",
    "Matches", "Season"
]

standard_exclude = [
    "Rk", "Player", "Nation", "Pos", "Squad", "Comp", "Season"
]

In [69]:
# Usage example:
standard_df_cleaned = clean_and_assign_headers(
    standard_df,
    standard_headers,
    columns_to_clean=['Squad', 'Comp', 'Nation'],
    drop_columns=['Matches'],
    exclude_numeric=standard_exclude
)

print(standard_df_cleaned.head())

  Rk               Player Nation    Pos           Squad            Comp   Age  \
0  1  Patrick van Aanholt    NED     DF  Crystal Palace  Premier League  26.0   
1  2       Rolando Aarons    ENG  MF,FW   Newcastle Utd  Premier League  21.0   
2  3       Rolando Aarons    ENG  MF,FW   Hellas Verona         Serie A  21.0   
3  4        Ignazio Abate    ITA     DF           Milan         Serie A  30.0   
4  5      Aymen Abdennour    TUN     DF       Marseille         Ligue 1  27.0   

   Born    MP  Starts  ...  xAG_per90  xG+xAG_per90  npxG_per90  \
0  1990  28.0    25.0  ...       0.04          0.25        0.21   
1  1995   4.0     1.0  ...       0.00          0.00        0.00   
2  1995  11.0     6.0  ...       0.00          0.00        0.00   
3  1986  17.0    11.0  ...       0.00          0.09        0.09   
4  1989   8.0     6.0  ...       0.00          0.00        0.00   

   npxG+xAG_per90  Gls_per90  Ast_per90  G+A_per90  G-PK_per90  G+A-PK_per90  \
0            0.25       0.13  

In [70]:
# Impute Christian Rutjens (row 2691)
mask = (standard_df_cleaned['Player'] == 'Christian Rutjens') & (standard_df_cleaned['Squad'] == 'Benevento')
standard_df_cleaned.loc[mask, 'Born'] = 1998
standard_df_cleaned.loc[mask, 'Age'] = 19

# Impute Max Moerstedt (row 18495)
mask = (standard_df_cleaned['Player'] == 'Max Moerstedt') & (standard_df_cleaned['Squad'] == 'Hoffenheim')
standard_df_cleaned.loc[mask, 'Born'] = 2005
standard_df_cleaned.loc[mask, 'Age'] = 18

# Impute Marco Pellegrino (rows 18785, 18786)
mask = (standard_df_cleaned['Player'] == 'Marco Pellegrino')
standard_df_cleaned.loc[mask, 'Born'] = 2002
standard_df_cleaned.loc[mask, 'Age'] = 21

# Impute Max Svensson (row 19240)
mask = (standard_df_cleaned['Player'] == 'Max Svensson') & (standard_df_cleaned['Squad'] == 'Osasuna')
standard_df_cleaned.loc[mask, 'Born'] = 1998
standard_df_cleaned.loc[mask, 'Age'] = 24

## Cleaning seasons_wages_df

In [71]:
seasons_wages_df.head()

,Rk,Squad,Competition,# of Players,Weekly Wages,Annual Wages,% Estimated,Season
0,1,Barcelona,es La Liga,33,"€ 4,682,115 (£ 3,925,993, $4,771,599)","€ 243,470,000 (£ 204,151,634, $248,123,125)",100%,2017-2018
1,2,Real Madrid,es La Liga,33,"€ 3,949,904 (£ 3,312,027, $4,025,393)","€ 205,395,000 (£ 172,225,426, $209,320,446)",100%,2017-2018
2,3,Paris S-G,fr Ligue 1,35,"€ 3,895,462 (£ 3,266,377, $3,969,910)","€ 202,564,000 (£ 169,851,608, $206,435,344)",100%,2017-2018
3,4,Manchester Utd,eng Premier League,35,"€ 3,810,955 (£ 3,195,596, $3,883,356)","€ 198,169,670 (£ 166,171,000, $201,934,520)",100%,2017-2018
4,5,Arsenal,eng Premier League,45,"€ 3,629,433 (£ 3,043,385, $3,698,385)","€ 188,730,521 (£ 158,256,000, $192,316,043)",100%,2017-2018


In [72]:
def clean_comp_column(df):
    """Clean and rename the Competition column to Comp."""
    df['Competition'] = df['Competition'].apply(keep_after_first_capital)
    df = df.rename(columns={'Competition': 'Comp'})
    return df

In [73]:
def extract_currency_values(s):
    """
    Extracts EUR, GBP, and USD values from a string like:
    '€ 4,682,115 (£ 3,925,993, $4,771,599)'
    Returns a dict: {'EUR': float, 'GBP': float, 'USD': float}
    """
    if pd.isnull(s):
        return {'EUR': None, 'GBP': None, 'USD': None}
    s = str(s)
    eur = gbp = usd = None
    eur_match = re.search(r'€\s?([\d,]+)', s)
    gbp_match = re.search(r'£\s?([\d,]+)', s)
    usd_match = re.search(r'\$\s?([\d,]+)', s)
    if eur_match:
        eur = float(eur_match.group(1).replace(',', ''))
    if gbp_match:
        gbp = float(gbp_match.group(1).replace(',', ''))
    if usd_match:
        usd = float(usd_match.group(1).replace(',', ''))
    return {'EUR': eur, 'GBP': gbp, 'USD': usd}

In [74]:
def add_wage_columns(df, wage_col, prefix):
    """Adds columns for EUR, GBP, and USD for a given wage column."""
    extracted = df[wage_col].apply(extract_currency_values)
    df[f'{prefix} Wages EUR'] = extracted.apply(lambda x: x['EUR'])
    df[f'{prefix} Wages GBP'] = extracted.apply(lambda x: x['GBP'])
    df[f'{prefix} Wages USD'] = extracted.apply(lambda x: x['USD'])
    return df

In [75]:
def process_seasons_wages(df, eur_to_usd=1.08, gbp_to_usd=1.27):
    """
    Cleans the seasons_wages DataFrame:
    - Cleans and renames the competition column
    - Extracts EUR, GBP, and USD values for weekly and annual wages
    - Stores exchange rates as attributes for reference
    Returns the cleaned DataFrame.
    """
    df = clean_comp_column(df)
    df = add_wage_columns(df, 'Weekly Wages', 'Weekly')
    df = add_wage_columns(df, 'Annual Wages', 'Annual')
    # Optionally, drop the original wage columns if you only want the extracted ones:
    # df = df.drop(columns=['Weekly Wages', 'Annual Wages'])
    # Store exchange rates as attributes for reference
    df.attrs['EUR_to_USD'] = eur_to_usd
    df.attrs['GBP_to_USD'] = gbp_to_usd
    return df

In [76]:
seasons_wages_cleaned = process_seasons_wages(seasons_wages_df)
print(seasons_wages_cleaned.head())
print("EUR to USD rate:", seasons_wages_cleaned.attrs['EUR_to_USD'])
print("GBP to USD rate:", seasons_wages_cleaned.attrs['GBP_to_USD'])

   Rk           Squad            Comp  # of Players  \
0   1       Barcelona         La Liga            33   
1   2     Real Madrid         La Liga            33   
2   3       Paris S-G         Ligue 1            35   
3   4  Manchester Utd  Premier League            35   
4   5         Arsenal  Premier League            45   

                            Weekly Wages  \
0  € 4,682,115 (£ 3,925,993, $4,771,599)   
1  € 3,949,904 (£ 3,312,027, $4,025,393)   
2  € 3,895,462 (£ 3,266,377, $3,969,910)   
3  € 3,810,955 (£ 3,195,596, $3,883,356)   
4  € 3,629,433 (£ 3,043,385, $3,698,385)   

                                  Annual Wages % Estimated     Season  \
0  € 243,470,000 (£ 204,151,634, $248,123,125)        100%  2017-2018   
1  € 205,395,000 (£ 172,225,426, $209,320,446)        100%  2017-2018   
2  € 202,564,000 (£ 169,851,608, $206,435,344)        100%  2017-2018   
3  € 198,169,670 (£ 166,171,000, $201,934,520)        100%  2017-2018   
4  € 188,730,521 (£ 158,256,000, $192,3

In [77]:
def add_position_onehot(df, pos_col='Pos'):
    """
    Adds one-hot columns for Defender, Midfielder, Forward, GK based on the position column.
    Handles multiple positions per player (e.g., 'MF,FW').
    """
    # Mapping from code to column name
    role_map = {
        'DF': 'Defender',
        'MF': 'Midfielder',
        'FW': 'Forward',
        'GK': 'GK'
    }
    # Ensure the column exists
    if pos_col not in df.columns:
        return df
    # Fill NaN with empty string to avoid errors
    df[pos_col] = df[pos_col].fillna('')
    for code, col in role_map.items():
        df[col] = df[pos_col].apply(lambda x: int(code in str(x).split(',')))
    return df

In [78]:
standard_df_cleaned = add_position_onehot(standard_df_cleaned, pos_col='Pos')
passing_df_cleaned = add_position_onehot(passing_df_cleaned, pos_col='Pos')
defending_df_cleaned = add_position_onehot(defending_df_cleaned, pos_col='Pos')

passing_df_cleaned.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Cmp_Total,...,Cmp_Medium/90,Cmp_Long/90,Att_Total/90,Att_Short/90,Att_Medium/90,Att_Long/90,Defender,Midfielder,Forward,GK
0,1.0,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,26.0,1990,24.3,884.0,...,12.80,3.05,48.40,21.89,17.74,6.26,1,0,0,0
2,3.0,Rolando Aarons,ENG,"MF,FW",Hellas Verona,Serie A,21.0,1995,5.7,87.0,...,4.39,0.70,21.05,10.53,6.14,1.40,0,1,1,0
3,4.0,Ignazio Abate,ITA,DF,Milan,Serie A,30.0,1986,11.7,625.0,...,24.53,4.96,66.32,25.13,28.46,10.26,1,0,0,0
4,5.0,Aymen Abdennour,TUN,DF,Marseille,Ligue 1,27.0,1989,5.5,310.0,...,26.91,4.36,60.55,23.27,28.18,6.18,1,0,0,0
6,7.0,Mehdi Abeid,ALG,MF,Dijon,Ligue 1,24.0,1992,13.1,513.0,...,16.72,3.36,46.18,20.15,18.70,4.73,0,1,0,0


In [79]:
# Save the cleaned DataFrames
defending_df_cleaned.to_csv("cleaned_data/defending_cleaned.csv", index=False)
passing_df_cleaned.to_csv("cleaned_data/passing_cleaned.csv", index=False)
seasons_stats_cleaned.to_csv("cleaned_data/seasons_stats_cleaned.csv", index=False)
standard_df_cleaned.to_csv("cleaned_data/standard_cleaned.csv", index=False)
seasons_wages_cleaned.to_csv("cleaned_data/seasons_wages_cleaned.csv", index=False)
shooting_df_cleaned.to_csv("cleaned_data/shooting_cleaned.csv", index=False)

## Some tests I ran throughout initial cleaning

In [80]:
# Test for Defending Data
def test_defending_df(defending_df_cleaned):
    print("Testing Defending DataFrame...")
    # Check 'Season' column existsKakarot12*
    assert "Season" in defending_df_cleaned.columns, "Season column missing in defending_df_cleaned"
    # Check for unique columns
    assert defending_df_cleaned.columns.is_unique, "Defending DataFrame columns are not unique"
    print("Defending DataFrame columns:", list(defending_df_cleaned.columns))
    print("Defending DataFrame test passed!\n")

# Test for Passing Data
def test_passing_df(passing_df_cleaned):
    print("Testing Passing DataFrame...")
    # Check 'Season' column exists
    assert "Season" in passing_df_cleaned.columns, "Season column missing in passing_df_cleaned"
    # Check for renamed duplicate columns
    expected_cols = [
        "Cmp_Total", "Cmp_Short", "Cmp_Medium", "Cmp_Long",
        "Att_Total", "Att_Short", "Att_Medium", "Att_Long",
        "Cmp%_Total", "Cmp%_Short", "Cmp%_Medium", "Cmp%_Long"
    ]
    for col in expected_cols:
        assert col in passing_df_cleaned.columns, f"{col} missing in passing_df_cleaned"
    print("Passing DataFrame columns:", [col for col in passing_df_cleaned.columns if "Cmp" in col or "Att" in col])
    print("Passing DataFrame test passed!\n")

# Run the tests
test_defending_df(defending_df_cleaned)
test_passing_df(passing_df_cleaned)

Testing Defending DataFrame...
Defending DataFrame columns: ['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s', 'Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'DribTkl', 'Att', 'Tkl%', 'Lost', 'Blocks', 'Sh', 'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Season', 'Interceptions/90', 'Tackles/90', 'Defender', 'Midfielder', 'Forward', 'GK']
Defending DataFrame test passed!

Testing Passing DataFrame...
Passing DataFrame columns: ['Cmp_Total', 'Att_Total', 'Cmp%_Total', 'Cmp_Short', 'Att_Short', 'Cmp%_Short', 'Cmp_Medium', 'Att_Medium', 'Cmp%_Medium', 'Cmp_Long', 'Att_Long', 'Cmp%_Long', 'Cmp_Total/90', 'Cmp_Short/90', 'Cmp_Medium/90', 'Cmp_Long/90', 'Att_Total/90', 'Att_Short/90', 'Att_Medium/90', 'Att_Long/90']
Passing DataFrame test passed!



In [81]:
numeric_df = shooting_df_cleaned.select_dtypes(include=[np.number]).astype(float)

print("Total NaNs:", numeric_df.isnull().sum().sum())
print("Total inf:", np.isinf(numeric_df.values).sum())
print("Total +inf:", (numeric_df.values == np.inf).sum())
print("Total -inf:", (numeric_df.values == -np.inf).sum())

print("Columns with inf:", numeric_df.columns[np.isinf(numeric_df).any()].tolist())
print("Columns with -inf:", numeric_df.columns[(numeric_df == -np.inf).any()].tolist())

Total NaNs: 6488
Total inf: 0
Total +inf: 0
Total -inf: 0
Columns with inf: []
Columns with -inf: []


In [82]:
nan_counts = shooting_df_cleaned.isnull().sum()
print("NaN counts per column:")
print(nan_counts[nan_counts > 0])

NaN counts per column:
SoT%       1072
G/Sh       1072
G/SoT      2200
Dist       1072
npxG/Sh    1072
dtype: int64


In [83]:
# Show rows with the most NaNs (e.g., top 20)
nan_counts_per_row = shooting_df_cleaned.isnull().sum(axis=1)
print("Rows with the most NaNs:")
print(shooting_df_cleaned.loc[nan_counts_per_row.sort_values(ascending=False).index[:20]])

# Alternatively, show all rows where any column is NaN
print("Rows with any NaNs:")
print(shooting_df_cleaned[shooting_df_cleaned.isnull().any(axis=1)].head(20))

# Or, to see the first few rows regardless
print("First 10 rows of cleaned data:")
print(shooting_df_cleaned.head(10))

Rows with the most NaNs:
           Rk                  Player Nation Pos            Squad  \
18611  1900.0          Moritz Nicolas    GER  GK         Gladbach   
17428   717.0      Stole Dimitrievski    MKD  GK   Rayo Vallecano   
11059   158.0         Daniel Bachmann    AUT  GK          Watford   
1039   1040.0        Samir Handanović    SVN  GK            Inter   
14690   868.0        Łukasz Fabiański    POL  GK         West Ham   
3080    389.0    Marc-Aurèle Caillard    FRA  GK         Guingamp   
9093   1014.0            Alfred Gomis    SEN  GK            Dijon   
6364   1017.0          Vicente Guaita    ESP  GK   Crystal Palace   
18639  1928.0            Ørjan Nyland    NOR  GK          Sevilla   
1045   1046.0                Joe Hart    ENG  GK         West Ham   
15576  1754.0           Birger Meling    NOR  DF           Rennes   
9638   1559.0            Mike Maignan    FRA  GK            Lille   
8649    570.0  Angelo da Costa Júnior    BRA  GK          Bologna   
5473    1

In [84]:
# Assume df is your DataFrame (e.g., standard_df_cleaned)
nan_rows = shooting_df_cleaned[shooting_df_cleaned.isnull().any(axis=1)]

# For each row with NaNs, print the row index, columns with NaNs, and the values
for idx, row in nan_rows.iterrows():
    nan_cols = row.index[row.isnull()].tolist()
    print(f"\nRow {idx} has NaNs in columns: {nan_cols}")
    print(row[nan_cols])
    print("Full row:")
    print(row)


Row 2 has NaNs in columns: ['G/SoT']
G/SoT    NaN
Name: 2, dtype: object
Full row:
Rk                    3.0
Player     Rolando Aarons
Nation                ENG
Pos                 MF,FW
Squad       Hellas Verona
Comp              Serie A
Age                  21.0
Born                 1995
90s                   5.7
Gls                   0.0
Sh                    3.0
SoT                   0.0
SoT%                  0.0
Sh/90                0.52
SoT/90                0.0
G/Sh                  0.0
G/SoT                 NaN
Dist                 17.4
FK                    0.0
PK                    0.0
PKatt                 0.0
xG                    0.2
npxG                  0.2
npxG/Sh              0.05
G-xG                 -0.2
np:G-xG              -0.2
Season          2017-2018
Name: 2, dtype: object

Row 9 has NaNs in columns: ['G/SoT']
G/SoT    NaN
Name: 9, dtype: object
Full row:
Rk                 10.0
Player     Amir Abrashi
Nation              ALB
Pos                  MF
Squad      

In [85]:
# Find and display all rows where Age or Born is NaN
nan_rows = passing_df_cleaned[passing_df_cleaned['Age'].isnull() | passing_df_cleaned['Born'].isnull()]
print("Rows with NaN in Age or Born:")
print(nan_rows)

Rows with NaN in Age or Born:
Empty DataFrame
Columns: [Rk, Player, Nation, Pos, Squad, Comp, Age, Born, 90s, Cmp_Total, Att_Total, Cmp%_Total, TotDist, PrgDist, Cmp_Short, Att_Short, Cmp%_Short, Cmp_Medium, Att_Medium, Cmp%_Medium, Cmp_Long, Att_Long, Cmp%_Long, Ast, xAG, xA, A-xAG, KP, PassesInFinal3rd, PPA, CrsPA, PrgP, Season, Cmp_Total/90, Cmp_Short/90, Cmp_Medium/90, Cmp_Long/90, Att_Total/90, Att_Short/90, Att_Medium/90, Att_Long/90, Defender, Midfielder, Forward, GK]
Index: []

[0 rows x 45 columns]
